# Experimento 4
Este experimento fue principalmente para verificar que pasa cuando no se tom unicamente el valor de tf-idf más grande, esto es por que en el experimento 3 se omitian estos valores, con el siguiente código vamos a obtener los vectores que vienen dados por la vectorización directa de los documentos usando tf-idf.

In [1]:
import sys,traceback
import unicodedata
import pickle
import numpy as np
from scipy.spatial import distance
import time
from math import exp,log

def tf(doc,p): #Código para obtener tf standard
	return doc.count(p)/len(doc)

def idf(docs,p,th): #código para obtener idf standard, con una restricción
	cont=0
	for d in docs:
		if p in d:
			cont+=1
	if cont<th:
		return 0
	else:
		return log(len(docs)/cont,10)

def splitInv(cad,caracteres,stopwords): 
    #split inverso que divide una cadena en subcadenas cuando
    #encuentra un caracter que no este en el set caracteres y
    #detiene palabras que esten en el set stopwords
	arreglo=[]
	aux=""
	for c in cad:
		if c not in caracteres :
			if aux.rstrip('sao')!="" and aux not in stopwords:
				arreglo.append(aux.rstrip('sao'))
			aux=""
		else:
			aux+=c
	# print(arreglo)
	return arreglo

def trnDocs(rawdocs,stopwords): 
    #transforma todos los documentos crudos a listas de palabras y detiene palabras
	docs={}
	for e,d in rawdocs.items():
		docs[e]=splitInv(unicodedata.normalize('NFKD',d),"qwertyuiopasdfghjklzxcvbnm",stopwords)
	return docs

def getVoc(docs,th=1,th2=0.1):
    #encuentra el vocabulario, usa restricciones para saber si acepta la palabra o no
	vocab=[]
	for d in docs:
		for p in d:
			if p not in [v[0] for v in vocab]:
				aux=idf(docs,p,th)
				# print(aux)
				if aux>=th2:
					vocab.append((p,aux))
	return vocab

def leer(corpus):
    #lee el corpus
	with open(corpus,"r") as f:
		for line in f:
			yield line.split("|",1)


def doc2Vec(doc,vocab,funcion):
    #transforma los documentos a vectores
	with open("../Experimento3/stopwords-es-master/stopwords-es.txt","r") as f:
	    stopwords=f.read().splitlines()
	try:
		palabras=splitInv(unicodedata.normalize('NFKD',doc),"qwertyuiopasdfghjklzxcvbnm",stopwords)
	except Exception as e:
		print(e,end="\r")
		palabras=doc
	vector=[]
	for key,value in vocab:
		vector.append(tf(palabras,key)*funcion(value)) #aqui probé diferentes funciones, pero daban resultados similares
	# print(vocab)
	return np.array(vector).reshape(1,-1)

def crearVectores(archCorpus,archPickle,funcion):
    #Esta función crea los vectores y los guarda en un pickle
	with open("../Experimento3/stopwords-es-master/stopwords-es.txt","r") as f:
	    stopwords=f.read().splitlines()
	ti=time.time()
	corpus=dict(leer(archCorpus))
	print("Corpus leido")
	docs=trnDocs(corpus,stopwords)
	print("Documentos normalizados")
	vocab=getVoc(docs.values(),2,0.1)
	print("Vocabulario obtenido")
	vectores=[(n,doc2Vec(d,vocab,funcion)) for n,d in docs.items()]
	print("Vectores obtenidos")
	print("Me tarde : ",time.time()-ti)
	pickle.dump([vectores,vocab],open(archPickle,"wb"))
	return [vectores,vocab]

def leerVectores(archPickle):
	return pickle.load(open(archPickle,"rb"))


def predecir(archSintomas,vectores,vocab,funcion):
	sintomas=open(archSintomas,"r").read()
	vector=doc2Vec(sintomas,vocab,funcion)
	docs=[]
	for n,v in vectores:
		# docs.append([n,distance.cdist( vector,v, 'wminkowski',w=np.random.rand(v.shape[0]))])
		# docs.append([n,distance.cdist( vector,v, 'matching')])
		# docs.append([n,distance.cdist( vector,v, 'braycurtis')])
		# docs.append([n,distance.cdist( vector,v, 'canberra')])
		# docs.append([n,distance.cdist( vector,v, 'chebyshev')])
		# docs.append([n,distance.cdist( vector,v, 'jaccard')])
		# docs.append([n,distance.cdist( vector,v, 'correlation')])
# 		docs.append([n,distance.cdist( vector,v, 'sqeuclidean')])
# 		docs.append([n,distance.cdist( vector,v, 'cityblock')])
		docs.append([n,distance.cdist( vector,v, 'cosine')])
# 		docs.append([n,distance.cdist( vector,v, 'euclidean')])
	docs.sort(key=lambda x:x[1])
	return docs

Para ejecutar el código usamos el siguiente pedazo de código

In [2]:
# funcion=lambda x:1/(1+exp(5*(1-x)))
funcion=lambda x:x
args=["datosLimpios.csv","vectoresExp5.pkl","misSintomas.txt"] #ejecución si no tenemos los vectores
vectores,vocab=crearVectores(args[0],args[1],funcion)
dists=predecir(args[2],vectores,vocab,funcion)
for d in range(5):
	print(dists[d])
#esto tarda como 3 min

Corpus leido
Documentos normalizados
Vocabulario obtenido
Vectores obtenidosnt 2 must be str, not list
Me tarde :  142.5667552947998
['dolor_espalda.html ', array([[0.78032679]])]
['sacralgia.html ', array([[0.85325697]])]
['artritis_reumatoide.html ', array([[0.88211162]])]
['espondilitis_anquilosante.html ', array([[0.88287248]])]
['ojo_seco.html ', array([[0.89080011]])]


In [3]:
args=["vectoresExp5.pkl","misSintomas.txt"]
vectores,vocab=leerVectores(args[0])
dists=predecir(args[1],vectores,vocab,funcion)
for d in range(15):
	print(dists[d])

['dolor_espalda.html ', array([[0.78032679]])]
['sacralgia.html ', array([[0.85325697]])]
['artritis_reumatoide.html ', array([[0.88211162]])]
['espondilitis_anquilosante.html ', array([[0.88287248]])]
['ojo_seco.html ', array([[0.89080011]])]
['estrabismo.html ', array([[0.8949926]])]
['ciatica.html ', array([[0.89587261]])]
['cervicitis.html ', array([[0.89873307]])]
['hipermetropia.html ', array([[0.90913063]])]
['hipotension.html ', array([[0.91832107]])]
['enfermedad_mano_pie_boca.html ', array([[0.91973343]])]
['sindrome_tunel_carpiano.html ', array([[0.92180831]])]
['torticolis.html ', array([[0.92193407]])]
['cancer-cabeza-cuello.html ', array([[0.92240652]])]
['dismenorrea.html ', array([[0.92493556]])]


## Conclusión
Nos da resultados similares a los que se habian obtenido con el Experimento 3, sin embargo los sintomas que fueron probados, corresponden más con la espondilitis anquilosante, 

In [4]:
args=["vectoresExp5.pkl","misSintomas2.txt"] #sintomas alergia
dists=predecir(args[1],vectores,vocab,funcion)
for d in range(15):
	print(dists[d])

['fractura_hueso_fatiga.html ', array([[0.81288285]])]
['hirsutismo.html ', array([[0.81616539]])]
['tos_ferina.html ', array([[0.82460046]])]
['sindrome-de-rett.html ', array([[0.85825827]])]
['asma.html ', array([[0.86753488]])]
['sindrome-hiperestimulacion-ovarica.html ', array([[0.90442022]])]
['sindrome-de-fatiga-cronica.html ', array([[0.91539611]])]
['silicosis.html ', array([[0.92790926]])]
['miastenia_gravis.html ', array([[0.93204193]])]
['insuficiencia-cardiaca.html ', array([[0.93245794]])]
['edema_pulmonar.html ', array([[0.9424433]])]
['sudor_de_manos.html ', array([[0.94917332]])]
['paperas.html ', array([[0.95056747]])]
['cancer_ovario.html ', array([[0.95751384]])]
['epoc.html ', array([[0.95905397]])]


In [5]:
args=["vectoresExp5.pkl","misSintomas3.txt"] # sintomas gripe
dists=predecir(args[1],vectores,vocab,funcion)
for d in range(15):
	print(dists[d])

['fractura_hueso_fatiga.html ', array([[0.81288285]])]
['hirsutismo.html ', array([[0.81616539]])]
['tos_ferina.html ', array([[0.82460046]])]
['sindrome-de-rett.html ', array([[0.85825827]])]
['asma.html ', array([[0.86753488]])]
['sindrome-hiperestimulacion-ovarica.html ', array([[0.90442022]])]
['sindrome-de-fatiga-cronica.html ', array([[0.91539611]])]
['silicosis.html ', array([[0.92790926]])]
['miastenia_gravis.html ', array([[0.93204193]])]
['insuficiencia-cardiaca.html ', array([[0.93245794]])]
['edema_pulmonar.html ', array([[0.9424433]])]
['sudor_de_manos.html ', array([[0.94917332]])]
['paperas.html ', array([[0.95056747]])]
['cancer_ovario.html ', array([[0.95751384]])]
['epoc.html ', array([[0.95905397]])]


In [6]:
args=["vectoresExp5.pkl","prueba.txt"] #sintomas gripe de otra pagina
dists=predecir(args[1],vectores,vocab,funcion)
for d in range(15):
	print(dists[d])

['gripe_porcina.html ', array([[0.71997584]])]
['gripe.html ', array([[0.72832763]])]
['otitis_media.html ', array([[0.80296828]])]
['escarlatina.html ', array([[0.84835831]])]
['otitis_externa.html ', array([[0.86049732]])]
['fiebre-tifoidea.html ', array([[0.86639654]])]
['malaria.html ', array([[0.8679358]])]
['legionela.html ', array([[0.87648166]])]
['infeccion_rotavirus.html ', array([[0.8843038]])]
['resfriado.html ', array([[0.89707004]])]
['difteria.html ', array([[0.89824274]])]
['bronquiolitis.html ', array([[0.8997844]])]
['colera.html ', array([[0.90032382]])]
['erisipela.html ', array([[0.90387189]])]
['faringitis.html ', array([[0.90976491]])]


Como conclusión puedo decir que este sistema funciona mejor para sintomas con descripciones lo suficientemente grandes, sin embargo a mi parecer tiene un desempeño similar al Experimento 4.